The  data  is  available  as  two  attached  CSV  files:
takehome_user_engagement. csv
takehome_users . csv
The  data  has  the  following  two  tables:<br>
1]  A  user  table  ( "takehome_users" )  with  data  on  12,000  users  who  signed  up  for  the
product  in  the  last  two  years.   This  table  includes:<br>
> ● name:  the  user's  name<br>
● object_id:   the  user's  id<br>
● email:  email  address<br>
● creation_source:   how  their  account  was  created. <br>
>>This  takes  on  one
of  5  values:<br>
○ PERSONAL_PROJECTS:  invited  to  join  another  user's
personal  workspace<br>
○ GUEST_INVITE:  invited  to  an  organization  as  a  guest
(limited  permissions)<br>
○ ORG_INVITE:  invited  to  an  organization  (as  a  full  member)<br>
○ SIGNUP:  signed  up  via  the  website<br>
○ SIGNUP_GOOGLE_AUTH:  signed  up  using  Google
Authentication  (using  a  Google  email  account  for  their  login
id)<br>

>● creation_time:  when  they  created  their  account<br>
● last_session_creation_time:   unix  timestamp  of  last  login<br>
● opted_in_to_mailing_list:  whether  they  have  opted  into  receiving
marketing  emails<br>
● enabled_for_marketing_drip:  whether  they  are  on  the  regular
marketing  email  drip<br>
● org_id:   the  organization  (group  of  users)  they  belong  to<br>
● invited_by_user_id:   which  user  invited  them  to  join  (if  applicable).<br><br>

2]  A  usage  summary  table  ( "takehome_user_engagement" )  that  has  a  row  for  each  day
that  a  user  logged  into  the  product.
Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate
days  in  at  least  one  seven­day  period ,  **identify  which  factors  predict  future  user
adoption .**
We  suggest  spending  1_2  hours  on  this,  but  you're  welcome  to  spend  more  or  less.
Please  send  us  a  brief  writeup  of  your  findings  (the  more  concise,  the  better  ­­  no  more
than  one  page),  along  with  any  summary  tables,  graphs,  code,  or  queries  that  can  help
us  understand  your  approach.  Please  note  any  factors  you  considered  or  investigation
you  did,  even  if  they  did  not  pan  out.  Feel  free  to  identify  any  further  research  or  data
you  think  would  be  valuable.

**<font size='5'>1. Data Preprocessing</font>**

In [1]:
import pandas as pd
import datetime as dt

In [2]:
users = pd.read_csv('takehome_users.csv', encoding = 'latin-1', parse_dates=['creation_time'])
engagement = pd.read_csv('takehome_user_engagement.csv', parse_dates=['time_stamp'])

In [3]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [5]:
engagement.visited.value_counts()

1    207917
Name: visited, dtype: int64

The column **visited** can be dropped since it only have one value as 1.

In [6]:
engagement.drop('visited',axis = 1, inplace=True)
engagement

,time_stamp,user_id
0,2014-04-22 03:53:30,1
1,2013-11-15 03:45:04,2
2,2013-11-29 03:45:04,2
3,2013-12-09 03:45:04,2
4,2013-12-25 03:45:04,2
...,...,...
207912,2013-09-06 06:14:15,11996
207913,2013-01-15 18:28:37,11997
207914,2014-04-27 12:45:16,11998
207915,2012-06-02 11:55:59,11999


Then I filtered out the rows in engagement that has less than 3 activites.

In [7]:
new_engagement = engagement.groupby('user_id').filter(lambda x: len(x) >= 3)
new_engagement

,time_stamp,user_id
1,2013-11-15 03:45:04,2
2,2013-11-29 03:45:04,2
3,2013-12-09 03:45:04,2
4,2013-12-25 03:45:04,2
5,2013-12-31 03:45:04,2
...,...,...
207905,2014-04-20 14:22:45,11991
207906,2014-04-25 14:22:45,11991
207907,2014-04-28 14:22:45,11991
207908,2014-05-17 14:22:45,11991


I added **count** column so I could calculate the day difference in the next step.

In [8]:
new_engagement['count'] = new_engagement.groupby('user_id').cumcount()+1
#new_engagment.reset_index(inplace=True)
new_engagement

,time_stamp,user_id,count
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,2
3,2013-12-09 03:45:04,2,3
4,2013-12-25 03:45:04,2,4
5,2013-12-31 03:45:04,2,5
...,...,...,...
207905,2014-04-20 14:22:45,11991,1
207906,2014-04-25 14:22:45,11991,2
207907,2014-04-28 14:22:45,11991,3
207908,2014-05-17 14:22:45,11991,4


In [9]:
#Splitting up odd and even number 
even_en = new_engagement[new_engagement['count'] % 2 == 0]
even_en

,time_stamp,user_id,count
2,2013-11-29 03:45:04,2,2
4,2013-12-25 03:45:04,2,4
6,2014-01-08 03:45:04,2,6
8,2014-02-08 03:45:04,2,8
10,2014-02-13 03:45:04,2,10
...,...,...,...
207898,2014-05-23 11:04:47,11988,26
207900,2014-05-26 11:04:47,11988,28
207902,2014-06-01 11:04:47,11988,30
207906,2014-04-25 14:22:45,11991,2


In [10]:
#Calculate the date difference for ever 2 visits
even_en['delta'] = even_en.sort_values(['user_id','count']).groupby('user_id')['time_stamp'].diff()
even_en

C:\Users\weijung.tseng\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,time_stamp,user_id,count,delta
2,2013-11-29 03:45:04,2,2,NaT
4,2013-12-25 03:45:04,2,4,26 days
6,2014-01-08 03:45:04,2,6,14 days
8,2014-02-08 03:45:04,2,8,31 days
10,2014-02-13 03:45:04,2,10,5 days
...,...,...,...,...
207898,2014-05-23 11:04:47,11988,26,3 days
207900,2014-05-26 11:04:47,11988,28,3 days
207902,2014-06-01 11:04:47,11988,30,6 days
207906,2014-04-25 14:22:45,11991,2,NaT


In [11]:
# Only kept rows that has 7 or less days in delta
even7 = even_en[even_en.delta <= dt.timedelta(days = 7)]
even7

,time_stamp,user_id,count,delta
10,2014-02-13 03:45:04,2,10,5 days
27,2013-02-19 22:08:03,10,8,5 days
31,2013-03-05 22:08:03,10,12,4 days
35,2013-03-20 22:08:03,10,16,5 days
37,2013-03-26 22:08:03,10,18,6 days
...,...,...,...,...
207894,2014-05-18 11:04:47,11988,22,4 days
207896,2014-05-20 11:04:47,11988,24,2 days
207898,2014-05-23 11:04:47,11988,26,3 days
207900,2014-05-26 11:04:47,11988,28,3 days


In [12]:
# Applied the same process on even number on count
odd_en = new_engagement[new_engagement['count'] % 2 == 1]
odd_en['delta'] = odd_en.sort_values(['user_id','count']).groupby('user_id')['time_stamp'].diff()
odd7 = odd_en[odd_en.delta <= dt.timedelta(days = 7)]

C:\Users\weijung.tseng\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [43]:
#Created union of sets from odd and even df
user_id_set = set(odd7.user_id) | set(even7.user_id)

In [14]:
# Added label column in users df
users['label'] = users['object_id'].isin(user_id_set)
users

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,label
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False
...,...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0,False
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN,False
11997,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0,False
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN,False


last_session_creation_time column is composed of seconds, it needed to transform into regular datetime format

In [15]:
users['last_session_creation_time'] = users['last_session_creation_time'].map(lambda data: 
                                    dt.datetime.fromtimestamp(int(data)).strftime('%Y-%m-%d %H:%M:%S'), na_action='ignore')

In [16]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,label
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 22:53:30,1,0,11,10803.0,False
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0,True
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 18:14:52,0,0,94,1525.0,False
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 03:09:28,0,0,1,5151.0,False
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 04:14:20,0,0,193,5240.0,False


In [17]:
# Created 3 columns that can be useful in moddeling
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'])
users['active_days'] = (users['last_session_creation_time'] - users['creation_time']).dt.days
users['account_days'] = (max(users['creation_time']) - users['creation_time']).dt.days
users['email'] = [x.split('@')[1] for x in users.email]  
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,label,active_days,account_days
0,1,2014-04-22 03:53:30,Clausen August,yahoo.com,GUEST_INVITE,2014-04-21 22:53:30,1,0,11,10803.0,False,-1.0,38
1,2,2013-11-15 03:45:04,Poole Matthew,gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0,True,135.0,196
2,3,2013-03-19 23:14:52,Bottrill Mitchell,gustr.com,ORG_INVITE,2013-03-19 18:14:52,0,0,94,1525.0,False,-1.0,437
3,4,2013-05-21 08:09:28,Clausen Nicklas,yahoo.com,GUEST_INVITE,2013-05-22 03:09:28,0,0,1,5151.0,False,0.0,374
4,5,2013-01-17 10:14:20,Raw Grace,yahoo.com,GUEST_INVITE,2013-01-22 04:14:20,0,0,193,5240.0,False,4.0,498


In [18]:
users.email.value_counts().head(10)

gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
gustr.com         1179
hotmail.com       1165
rerwl.com            2
oqpze.com            2
qgjbc.com            2
dqwln.com            2
Name: email, dtype: int64

In [19]:
top_6_emails = users.email.value_counts().head(6)
users['email'] = [x if x in top_6_emails.keys() else 'other' for x in users.email]
#users['email'] = [x of ]

In [20]:
users.email.value_counts()

gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
other             1186
gustr.com         1179
hotmail.com       1165
Name: email, dtype: int64

In [21]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   name                        12000 non-null  object        
 3   email                       12000 non-null  object        
 4   creation_source             12000 non-null  object        
 5   last_session_creation_time  8823 non-null   datetime64[ns]
 6   opted_in_to_mailing_list    12000 non-null  int64         
 7   enabled_for_marketing_drip  12000 non-null  int64         
 8   org_id                      12000 non-null  int64         
 9   invited_by_user_id          6417 non-null   float64       
 10  label                       12000 non-null  bool          
 11  active_days                 8823 non-null   float64   

In [22]:
# Drop the columns that will not be needed
# org_id and invited_by_user_id has too many categories
users.drop(['object_id','name','org_id','creation_time','last_session_creation_time','invited_by_user_id'], axis = 1, inplace = True)

In [23]:
users.active_days.value_counts()

-1.0      3027
 0.0      1617
 1.0       822
 2.0       405
 3.0       195
          ... 
 551.0       1
 423.0       1
 175.0       1
 549.0       1
 424.0       1
Name: active_days, Length: 512, dtype: int64

In [24]:
print(users['label'][users['active_days'].isnull()].value_counts())
print(users['label'][users['active_days']<=0].value_counts())
# All the null value in active_days are false in label column, so I will change them all to 0. 
users['active_days'][users['active_days'].isnull()]

False    3177
Name: label, dtype: int64
False    4644
Name: label, dtype: int64


7       NaN
8       NaN
11      NaN
14      NaN
15      NaN
         ..
11975   NaN
11977   NaN
11984   NaN
11992   NaN
11993   NaN
Name: active_days, Length: 3177, dtype: float64

In [25]:
users['active_days'][users['active_days'].isnull()] = 0
users['active_days'][users['active_days'] <0 ] = 0

<ipython-input-25-cb4237f460b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['active_days'][users['active_days'].isnull()] = 0
<ipython-input-25-cb4237f460b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['active_days'][users['active_days'] <0 ] = 0


In [26]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   email                       12000 non-null  object 
 1   creation_source             12000 non-null  object 
 2   opted_in_to_mailing_list    12000 non-null  int64  
 3   enabled_for_marketing_drip  12000 non-null  int64  
 4   label                       12000 non-null  bool   
 5   active_days                 12000 non-null  float64
 6   account_days                12000 non-null  int64  
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 574.3+ KB


**<font size='5'>2. Predictive Modeling</font>**

In [27]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
# make dummy variable, and creat X & y
source = pd.get_dummies(users.creation_source, drop_first=True, prefix='source')
email = pd.get_dummies(users.email, drop_first=True, prefix='email')
model = pd.concat([users, source, email], axis = 1).drop(['creation_source', 'email'], axis = 1)
X = model.drop('label', axis = 1)
y = model['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [30]:
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('model', RandomForestClassifier(class_weight = 'balanced'))])
params = {'model__n_estimators':[10,30,50,100,150],
          'model__max_features':['auto', 'sqrt','log2'],
          'model__max_depth':[5,7,9] }
CV = GridSearchCV(pipeline, params, n_jobs=-1, cv = 10, scoring = 'accuracy', verbose = 1)
CV.fit(X_train, y_train)
print(CV.best_params_)

Fitting 10 folds for each of 45 candidates, totalling 450 fits
{'model__max_depth': 9, 'model__max_features': 'auto', 'model__n_estimators': 30}


In [31]:
# Use Best Parameters
RF_Pipeline = Pipeline([('scaler', StandardScaler()),
                         ('model', RandomForestClassifier(class_weight = 'balanced',
                                                          max_depth = 9,
                                                          max_features ='auto',
                                                          n_estimators = 30))])
RF_Pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=9,
                                        n_estimators=30))])

In [38]:
# Accuracy Evaluation
y_pred = RF_Pipeline.predict(X_test)
ac_score = accuracy_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred, average = 'macro' )
c_matrix = confusion_matrix(y_test, y_pred)
print('Accuracy_score:', ac_score)
print('F1_score:', f_score)
print('Confustion Matrix:', '\n', c_matrix)
print('Classification Report', '\n',classification_report(y_test, y_pred))

Accuracy_score: 0.9543333333333334
F1_score: 0.9108648387813857
Confustion Matrix: 
 [[2479  107]
 [  30  384]]
Classification Report 
               precision    recall  f1-score   support

       False       0.99      0.96      0.97      2586
        True       0.78      0.93      0.85       414

    accuracy                           0.95      3000
   macro avg       0.89      0.94      0.91      3000
weighted avg       0.96      0.95      0.96      3000



In [42]:
# Feature Importance Evaluation
Variables = pd.DataFrame({'Features': X_train.columns, 'weight':RF_Pipeline.named_steps.model.feature_importances_})
Variables.sort_values(by='weight', ascending = False)

,Features,weight
2,active_days,0.920364
3,account_days,0.051667
5,source_PERSONAL_PROJECTS,0.007391
0,opted_in_to_mailing_list,0.002840
1,enabled_for_marketing_drip,0.002791
4,source_ORG_INVITE,0.002399
13,email_yahoo.com,0.002397
8,email_gmail.com,0.002077
6,source_SIGNUP,0.001670
11,email_jourrapide.com,0.001401


We can see that the major feature is **active_days**, and **account_days** is the second import feature but it's much more less important than the prior feature. The rest of the feature are relaitvely lower than these two feature.
<br>To increase adoption rate, the company needs to primarly focus on **active_days**, which is the days difference from the date of last session and the date of account creation. My suggestion is to create incentives like discount coupons or small gifts for the clients to stay active. <br>
The fact that the other features are much more less important may contruibute to the definition of "adopted user", which is a user who has logged into the product on three separate days in at least one seven_day period. Statistically speaking, the longer a user has been active, the higher the chance that he is classified as an adopted user. However, this category excluded loyal users that logged twice a week and inclued user that only logged in 3 times in the first week but never logged back again. The purpose of this research is not clear but if the purpose is to identify loyal users, we will need a new definition that fits better.